# <font color=#f4665a>  ICA analysis </font>

### Project: BMPD HC
____________________________________________________

**Description:** This notebook provides code for BOLD signal fMRI resting-state processing for the Biomarker for Parkinson's Disease (BMPD)data. 
We will used ICA analysis:
CanICA is an ICA package for group-level analysis of fMRI data.  
It brings a well-controlled group model, as well as a thresholding algorithm controlling for specificity and sensitivity with an explicit model of the signal.  
The reference papers are: G. Varoquaux et al. "A group model for stable multi-subject ICA on fMRI datasets", NeuroImage Vol 51 (2010), p. 288-299
G. Varoquaux et al. "ICA-based sparse features recovery from fMRI datasets", IEEE ISBI 2010, p. 1177


**Toolbox required:** SpinalCordToolbox, FSL, nilearn toolbox, nipype, matlab

**Inputs**:  
This notebook required this the following prepross anatomical,fmri images 

**Ouputs**:
See the output description at each step of the Notebook.

____________________________________________________


## <font color=#00988c> Imports </font>

In [76]:
import sys
import json
# Spinal cord Toolbox_________________________________________
### Cerebro:
sys.path.append("/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/spinalcordtoolbox-5.0.0")
sys.path.append("/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/spinalcordtoolbox-5.0.0/scripts") #sys.path.insert(0, "/cerebro/cerebro1/dataset/bmpd/derivatives/sc_preproc/code/sct/spinalcordtoolbox")
sys.path.append("/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/hc_project_analyses/code/") #sys.path.insert(0, "/cerebro/cerebro1/dataset/bmpd/derivatives/sc_preproc/code/sct/spinalcordtoolbox")

from spinalcordtoolbox.utils.sys import run_proc

from canICA_analyses import ICA
%matplotlib inline
%load_ext autoreload
%autoreload 2


import matplotlib.pyplot as plt


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## <font color=#00988c>  Run the ICA analysis </font>

In [2]:
# Load the dataset config
#config_spine_only_CL.json #../config/config_brsc_CL.json
with open('../../config/config_canICA_brsc.json') as config_file:
    config = json.load(config_file)
dataset="mtl" 
structures=["brain","spinalcord"] # ["spinalcord"] or ["brain","spinalcord"] . double check the script for brainsc

In [3]:
import glob
files_func={};func_allsbj={}
for structure in structures:
    if len(structures) == 1:
        ana=structure
    else:
        ana= "brain_spinalcord"
    files_func[structure]=[];func_allsbj[structure]=[]
    for sbj_nb in range(len(config["list_subjects"][dataset])):
        subject_name=config["list_subjects"][dataset][sbj_nb]
        files_func[structure].append(glob.glob(config["data"][dataset]["inputs_ica"]["dir"]+ '/sub-' + subject_name + '/'  + structure + '/*' + config["data"][dataset]["inputs_ica"][structure]["tag_filename_" + ana] + '*')[0])
        

In [5]:
config["ica_ana"]["k_range"][ana]=[5]
for k in config["ica_ana"]["k_range"][ana]:
    config["ica_ana"]["n_comp"]=k # usefull if you want to test only on k
    print(config["ica_ana"]["n_comp"])
    if ana == "spinalcord" or ana=="brain":
        icas = ICA(files_func[structure],[''],structures,dataset,config) # "brain_spinalcord" or "brain" or "spinalcord"
    elif ana=="brain_spinalcord":
        icas = ICA(files_func[structures[0]],files_func[structures[1]],structures,dataset,config) # "brain_spinalcord" or "brain" or "spinalcord"
    
    all_data=icas.get_data(run='extract',t_r=config["acq_params"][dataset]["TR"],n_jobs=8) # load or extract, if NaN issues put both
    reducedata_all=icas.indiv_PCA(all_data,save_indiv_img=True) # that step is not implanted to save individual maps for brain + sc yet
    
    components=icas.get_CCA(reducedata_all)
    components_final,components_final_z=icas.get_ICA(components)
    zcomponents4D_filename=icas.save_components(components_final,components_final_z)
    

5
Analyse will be run on brain and spinalcord structures simultaneously
/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//hc_project_analyses/masks/brain_old_3mm/MNI_GM_3mm.nii
/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/hc_project_analyses/masks/spinalcord/PAM50_C1C7_gmwm.nii.gz
(265, 103503)
(265, 103503)
(265, 103503)
(265, 103503)
(265, 103503)
(265, 103503)
(265, 103503)
(265, 103503)
(265, 103503)
(265, 103503)
(265, 103503)
(265, 103503)
(265, 103503)
(265, 103503)
(265, 103503)
(265, 103503)
(265, 103503)
(265, 103503)
(265, 103503)
(265, 103503)
(265, 103503)
(265, 103503)
(265, 103503)
(265, 103503)
(265, 103503)
(265, 103503)
(265, 103503)
(265, 103503)
(265, 103503)
(265, 103503)
(265, 103503)
>> Individual PCA done <<


In [75]:
zcomponents4D_filename=icas.save_components(components_final,components_final_z)

>> Components z-scored done <<


In [73]:
components_z={}
analyse_dir=config["main_dir"] + config["data"][dataset]["ica"][structures[0]+'_'+structures[1]]["dir"]+'/' + '/' + '/K_' + str(config["ica_ana"]["n_comp"])

for structure in structures:
    components_z[structure]=np.zeros(shape=(components_final[structure].shape[0],components_final[structure].shape[1]),dtype='float')
    nifti_masker[structure]= NiftiMasker(mask_img=config["main_dir"] + config["masks"][dataset][structure], standardize=False,smoothing_fwhm=None).fit() #Extract the data inside the mask and create a vector
    for i in range(0,len(components_final[structure].T)):
        med  = np.median(components_final[structure])
        components_z[structure][:,i] = (components_final[structure][:,i]-med)/np.sqrt((np.sum((components_final[structure][:,i]-med)**2))/len(components_final[structure][:,i]))   # normalization 
        zcomponents_img = nifti_masker[structure].inverse_transform(components_z[structure].T) #check the component
        
        zcomponents4D_filename=analyse_dir  + '/comp_zscored/CanICA_' + str(len(config["list_subjects"][dataset])) + 'sbj_'+ structures[0] +'_'+ structure + '_K_'+ str(config["ica_ana"]["n_comp"]) + '_4D_z.nii.gz'
        zcomponents_img.to_filename(zcomponents4D_filename)

In [16]:
20*31
from nilearn.maskers import NiftiMasker
import numpy as np

In [43]:
print(reducedata_all[j-n_comp_pca:j,0:n_voxels[structures[0]]].shape)
print(reducedata_all[j-n_comp_pca:j,n_voxels[structures[1]]:].shape)
#n_voxels[structures[1]].shape
structure


(20, 50572)
(20, 50572)


'spinalcord'

In [46]:
nifti_masker={};n_voxels={};components_final={}
components_img={}
n_comp_pca=20
for structure in structures:
    nifti_masker[structure]= NiftiMasker(mask_img=config["main_dir"] + config["masks"][dataset][structure], standardize=False,smoothing_fwhm=None).fit() #Extract the data inside the mask and create a vector
    n_voxels[structure]= nifti_masker[structure].fit_transform(config["main_dir"] + config["masks"][dataset][structure]).shape[1] # number of voxels in the structure
print(n_voxels[structures[0]])
print(n_voxels[structures[1]])

50572
52931


In [52]:
analyse_dir=config["main_dir"] + config["data"][dataset]["ica"][structures[0]+'_'+structures[1]]["dir"]+'/' + '/' + '/K_' + str(config["ica_ana"]["n_comp"])
nifti_masker={};n_voxels={};components_final={}
components_img={}
n_comp_pca=20
for structure in structures:
    nifti_masker[structure]= NiftiMasker(mask_img=config["main_dir"] + config["masks"][dataset][structure], standardize=False,smoothing_fwhm=None).fit() #Extract the data inside the mask and create a vector
    n_voxels[structure]= nifti_masker[structure].fit_transform(config["main_dir"] + config["masks"][dataset][structure]).shape[1] # number of voxels in the structure
       
    j=0
    for sbj_nb in range(0,len(config["list_subjects"][dataset])):
        subject_name=config["list_subjects"][dataset][sbj_nb]
        for i in range(j,j+n_comp_pca):
            j=i
            j=+i+1
        
        if structure==structures[0]:
            components_img[structures[0]] = nifti_masker[structures[0]].inverse_transform(reducedata_all[j-n_comp_pca:j,0:n_voxels[structures[0]]]) # transform the components in nifti
            components_img[structures[0]].to_filename(analyse_dir + '/pca_indiv/sub-' + subject_name +'_'+structures[0]+'_comp_PCA.nii.gz') #save the n principal components for each subject
  
        else:
            components_img[structures[1]] = nifti_masker[structures[1]].inverse_transform(reducedata_all[j-n_comp_pca:j,n_voxels[structures[0]]:]) # transform the components in nifti
            components_img[structures[1]].to_filename(analyse_dir + '/pca_indiv/sub-' + subject_name +'_'+structures[1]+'_comp_PCA.nii.gz') #save the n principal components for each subject
  
                     

In [35]:
#reducedata_all[j-n_comp_pca:j,0:n_voxels[structure[0]]].shape
n_voxels[structure[0]]

KeyError: 'b'

In [ ]:
# save PCA

# For brain and spinal cord analyses split the component in two voxel matrices to be transform in two separates images in the next step
if len(structures_ana) == 2:
    n_voxels={};nifti_masker={}
    for structure in structures:
        nifti_masker[structure]= NiftiMasker(mask_img=config["main_dir"] + config["masks"][dataset][structure], standardize=False,smoothing_fwhm=None).fit() #Extract the data inside the mask and create a vector
        n_voxels[structure]= nifti_masker[structure].fit_transform(config["main_dir"] + config["masks"][self.dataset][structure]).shape[1] # number of voxels in the structure
        components_final[structure]=np.empty(shape=(n_voxels[structure],config["ica_ana"]["n_comp"]),dtype='float') # matrix of brain voxels 
        components_final_z[structure]=np.empty(shape=(n_voxels[structure],config["ica_ana"]["n_comp"]),dtype='float') # matrix of brain voxels 
            
        for voxel in range(0,n_voxels[structures[0]]):
            components_final[structures[0]][voxel,:]=components_[voxel,:]
            #components_final_z[self.structures[0]][voxel,:]=components_z[voxel,:]
        for voxel in range(n_voxels[structures[0]],n_voxels[structures[0]]+n_voxels[structures[1]]):
            components_final[structures[1]][voxel-n_voxels[structures[0]],:]=components_[voxel,:]
            #components_final_z[self.structures[1]][voxel-n_voxels[self.structures[0]],:]=components_z[voxel,:]
